# Recurrent Networks Example

Recurrent neural networks use self-connections to model temporal sequences. A single "recurrent layer" takes input from its previous state as well as some other input. The image below is from the blog post ["The Unreasonable Effectiveness of Recurrent Neural Networks"](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) by Andrej Karpathy, which explains a lot about RNNs.

![Recurrent Neural Network](images/karpathy_rnn.jpeg)

Not all of the above architectures are easily modeled in Keras (in particular, the second and fourth are tricky to do, and require some workarounds). From the [Keras RNN documentation](https://keras.io/layers/recurrent/), we see that an RNN layer has input dimensions `(nb_samples, timesteps, input_dim)` and output dimensions of either `(nb_samples, timesteps, output_dim)` if the `return_sequences` argument is set to `True`, or `(nb_samples, output_dim)` otherwise. The first behavior corresponds to the fifth figure, while the second behavior corresponds to the third figure.

## Building a simple RNN in Theano

To understand how Keras works, it is a good idea to start off with something we can understand from the inside out. Let's build a vanilla recurrent neural network in Theano. The internal state of the network is updated according to the equation:

```
new_hidden_state = tanh(dot(input_vector, W) + dot(prev_hidden, U) + b)
```

Where:
- `new_hidden_state` the updated state of the recurrent layer
- `tanh` the activation function (the new state should be a nonlinear transformation of a linear combination of the new input step and its previous state)
- `dot` the dot product between a vector and a matrix
- `input_vector` the input at one timestep
- `W` the input-to-hidden connection
- `prev_hidden` the previous hidden state
- `U` the hidden-to-hidden connection
- `b` the bias vector (similar to a normal dense layer, it is a good idea to have a bias vector)

As the number of variables in the equations gets bigger, it will become impractical to explain them all, but I've used a similar naming convention elsewhere in this notebook.

First, let's import the necessary packages, define a random number generator and get the datatype to use for all our matrices.

In [1]:
from __future__ import print_function

import theano
from theano import tensor as T
import numpy as np

np.random.seed(42)
rng = np.random.RandomState(1337)
dtype = theano.config.floatX

Next, let's make some helper methods for generating weights as shared variables, like we did with the XOR example.

In [2]:
def get_weights(n_in, n_out):
    mag = 4. * np.sqrt(6. / (n_in + n_out))
    W_value = np.asarray(rng.uniform(low=-mag, high=mag, size=(n_in, n_out)), dtype=dtype)
    W = theano.shared(value=W_value, name='W_%d_%d' % (n_in, n_out), borrow=True, strict=False)
    return W

def get_bias(n_out):
    b_value = np.asarray(np.zeros((n_out,), dtype=dtype), dtype=theano.config.floatX)
    b = theano.shared(value=b_value, name='b_%d' % n_out, borrow=True, strict=False)
    return b

In the Theano XOR example, we used a simple subgradient descent optimizer. **Subgradient descent** means that, for a sample input, we change the weights of the network in the direction that minimizes the error between the desired output and the actual output. However, a good deal of work has gone into improving on this simple concept. Without getting into the details, a good optimizer for recurrent networks is the **RMSprop** optimizer. In Keras, it is very easy to specify that we want to use this optimizer, but since we are implementing everything in Theano right now, we have to define it ourselves.

In [3]:
def rmsprop(cost, params, learning_rate, rho=0.9, epsilon=1e-6):
    updates = list()
    for param in params:
        accu = theano.shared(np.zeros(param.get_value(borrow=True).shape, dtype=dtype),
                             broadcastable=param.broadcastable)
        grad = T.grad(cost, param)
        accu_new = rho * accu + (1 - rho) * grad ** 2

        updates.append((accu, accu_new))
        updates.append((param, param - (learning_rate * grad / T.sqrt(accu_new + epsilon))))
    return updates

Now that we have these helper methods, let's write a framework for testing how a general network performs. This framework takes as inputs the number of input and output dimensions, the input tensor of the network, and the predicted output, plus the network parameters (weights).

In [4]:
def test(n_in, n_out, X, output, params):
    output = output[-1, :] # get the last timestep from the network
    y = T.matrix(name='y', dtype=dtype) # the target variable
    lr = T.scalar(name='lr', dtype=dtype) # the learning rate (as a variable we can change)

    # minimize binary crossentropy
    xent = -y * T.log(output) - (1 - y) * T.log(1 - output)
    cost = xent.mean()
    
    # use rmsprop to get the network updates
    updates = rmsprop(cost, params, lr)

    # generate our testing data
    t_sets = 10
    X_datas = [np.asarray(rng.rand(20, n_in) > 0.5, dtype=dtype) for _ in range(t_sets)]
    y_datas = [np.asarray(rng.rand(1, n_out) > 0.5, dtype=dtype) for _ in range(t_sets)]

    # theano functions for training and testing
    train = theano.function([X, y, lr], [cost], updates=updates)
    test = theano.function([X], [output])

    # some starting parameters
    l = 0.1
    n_train = 1000

    # calculate and display the cost
    cost = sum([train(X_data, y_data, 0)[0] for X_data, y_data in zip(X_datas, y_datas)])
    print('Before training:', cost)

    for i in range(n_train):
        for X_data, y_data in zip(X_datas, y_datas):
            train(X_data, y_data, l)

        if (i+1) % (n_train / 5) == 0:
            cost = sum([train(X_data, y_data, 0)[0] for X_data, y_data in zip(X_datas, y_datas)])
            print('%d (lr = %f):' % (i+1, l), cost)
            l *= 0.5

Now that we have these helper components, let's write the code for our basic RNN. This follows the equations seen above.

In [5]:
def generate_and_test_vanilla_rnn(n_in, n_hidden, n_out):
    X = T.matrix(name='X', dtype=dtype)

    # the weights being used in the network
    w_in = get_weights(n_in, n_hidden)
    w_hidden = get_weights(n_hidden, n_hidden)
    w_out = get_weights(n_hidden, n_out)

    # the biases
    b_hidden = get_bias(n_hidden)
    b_out = get_bias(n_out)
    h_0 = get_bias(n_hidden)
    
    # collect all the parameters here so we can pass them to the optimizer
    params = [w_in, b_hidden, w_out, b_out, w_hidden, h_0]
    
    # define the recurrence here
    def step(x_t, h_tm1):
        h_t = T.tanh(T.dot(x_t, w_in) + T.dot(h_tm1, w_hidden) + b_hidden)
        y_t = T.nnet.sigmoid(T.dot(h_t, w_out) + b_out)
        return h_t, y_t

    [_, output], _ = theano.scan(fn=step, sequences=X, outputs_info=[h_0, None], n_steps=X.shape[0])

    test(n_in, n_out, X, output, params)

generate_and_test_vanilla_rnn(10, 50, 1)

Before training: 15.9748073443
200 (lr = 0.100000): 13.5803246217
400 (lr = 0.050000): 9.68473999735
600 (lr = 0.025000): 7.76820204681
800 (lr = 0.012500): 6.94260957224
1000 (lr = 0.006250): 6.76449744031


## Solving Gradient Decay

The basic problem with simple RNNs is that they don't model long sequence very well. LSTMs and GRUs solve this problem. The graphic below was taken from [here](https://www.reddit.com/r/MachineLearning/comments/3vnwum/rnn_vs_lstm_vanishing_gradients/), and shows the magnitude of the gradients as we move backwards in time. As the graphic shows, the gradients of the basic RNN take only around 20 timesteps before they stop conveying information, while the LSTM gradients convey useful information even after 100 timesteps. In other words, **the effects of early inputs are much more pronounced in the LSTM network than the simple RNN network, so long time sequences are more easily differentiated**.

![RNN Gradients Comparison](images/rnn_grads.gif)

The LSTM network does this by using **gates**. Some of the information from the LSTM is siphoned into a separate state, where it sits around, unaffected by the network, before being let out later on. This is explained in good detail in ["Understanding LSTM Networks"](http://colah.github.io/posts/2015-08-Understanding-LSTMs/) by Christopher Olah, which shows these graphics:

### Basic RNN

![Basic RNN](images/basic_rnn_graphic.png)

### LSTM

![LSTM](images/lstm_graphic.png)

The equations defining an LSTM are as follow:

```
input_gate = tanh(dot(input_vector, W_input) + dot(prev_hidden, U_input) + b_input)
forget_gate = tanh(dot(input_vector, W_forget) + dot(prev_hidden, U_forget) + b_forget)
output_gate = tanh(dot(input_vector, W_output) + dot(prev_hidden, U_output) + b_output)

candidate_state = tanh(dot(x, W_hidden) + dot(prev_hidden, U_hidden) + b_hidden)
memory_unit = prev_candidate_state * forget_gate + candidate_state * input_gate

new_hidden_state = tanh(memory_unit) * output_gate
```

It is a good idea to try to understand these equations at a reasonable level, because they play a role in the next notebook.

Let's go ahead and implement this in Theano.

In [6]:
def generate_and_test_lstm(n_in, n_hidden, n_out):
    X = T.matrix(name='X', dtype=dtype)

    # there are a lot of parameters, so let's add them incrementally
    params = list()

    # input gate
    w_in_input = get_weights(n_in, n_hidden)
    w_hidden_input = get_weights(n_hidden, n_hidden)
    b_input = get_bias(n_hidden)
    params += [w_in_input, w_hidden_input, b_input]

    # forget gate
    w_in_forget = get_weights(n_in, n_hidden)
    w_hidden_forget = get_weights(n_hidden, n_hidden)
    b_forget = get_bias(n_hidden)
    params += [w_in_forget, w_hidden_forget, b_forget]

    # output gate
    w_in_output = get_weights(n_in, n_hidden)
    w_hidden_output = get_weights(n_hidden, n_hidden)
    b_output = get_bias(n_hidden)
    params += [w_in_output, w_hidden_output, b_output]

    # hidden state
    w_in_hidden = get_weights(n_in, n_hidden)
    w_hidden_hidden = get_weights(n_hidden, n_hidden)
    b_hidden = get_bias(n_hidden)
    params += [w_in_hidden, w_hidden_hidden, b_hidden]

    # output
    w_out = get_weights(n_hidden, n_out)
    b_out = get_bias(n_out)
    params += [w_out, b_out]

    # starting hidden and memory unit state
    h_0 = get_bias(n_hidden)
    c_0 = get_bias(n_hidden)
    params += [h_0, c_0]
    
    # define the recurrence here
    def step(x_t, h_tm1, c_tm1):
        input_gate = T.nnet.sigmoid(T.dot(x_t, w_in_input) + T.dot(h_tm1, w_hidden_input) + b_input)
        forget_gate = T.nnet.sigmoid(T.dot(x_t, w_in_forget) + T.dot(h_tm1, w_hidden_forget) + b_forget)
        output_gate = T.nnet.sigmoid(T.dot(x_t, w_in_output) + T.dot(h_tm1, w_hidden_output) + b_output)

        candidate_state = T.tanh(T.dot(x_t, w_in_hidden) + T.dot(h_tm1, w_hidden_hidden) + b_hidden)
        memory_unit = c_tm1 * forget_gate + candidate_state * input_gate

        h_t = T.tanh(memory_unit) * output_gate
        y_t = T.nnet.sigmoid(T.dot(h_t, w_out) + b_out)

        return h_t, memory_unit, y_t

    [_, _, output], _ = theano.scan(fn=step, sequences=X, outputs_info=[h_0, c_0, None], n_steps=X.shape[0])

    test(n_in, n_out, X, output, params)

generate_and_test_lstm(10, 50, 1)

Before training: 10.8128263049
200 (lr = 0.100000): 1.94271907179e-05
400 (lr = 0.050000): 1.16528041576e-05
600 (lr = 0.025000): 9.67879741156e-06
800 (lr = 0.012500): 8.92299780359e-06
1000 (lr = 0.006250): 8.58776548077e-06


## GRUs, for good measure

Since we've done LSTMs, we should do GRUs as well. Again from Christopher Olah's blog post, the GRU looks like:

![GRU](images/gru_graphic.png)

The equations are simpler than the LSTM equations, and look like:

```
update_gate = tanh(dot(input_vector, W_update) + dot(prev_hidden, U_update) + b_update)
reset_gate = tanh(dot(input_vector, W_reset) + dot(prev_hidden, U_reset) + b_reset)

reset_hidden = prev_hidden * reset_gate
temp_state = tanh(dot(input_vector, W_hidden) + dot(reset_hidden, U_reset) + b_hidden)
new_hidden_state = (1 - update_gate) * temp_state + update_gate * prev_hidden
```

Let's go ahead and implement this in Theano.

In [7]:
def generate_and_test_gru(n_in, n_hidden, n_out):
    X = T.matrix(name='X', dtype=dtype)

    # there are a lot of parameters, so let's add them incrementally
    params = list()

    # update gate
    w_in_update = get_weights(n_in, n_hidden)
    w_hidden_update = get_weights(n_hidden, n_hidden)
    b_update = get_bias(n_hidden)
    params += [w_in_update, w_hidden_update, b_update]

    # reset gate
    w_in_reset = get_weights(n_in, n_hidden)
    w_hidden_reset = get_weights(n_hidden, n_hidden)
    b_reset = get_bias(n_hidden)
    params += [w_in_reset, w_hidden_reset, b_reset]

    # hidden layer
    w_in_hidden = get_weights(n_in, n_hidden)
    w_reset_hidden = get_weights(n_hidden, n_hidden)
    b_in_hidden = get_bias(n_hidden)
    params += [w_in_hidden, w_reset_hidden, b_in_hidden]

    # output
    w_out = get_weights(n_hidden, n_out)
    b_out = get_bias(n_out)
    params += [w_out, b_out]

    # starting hidden state
    h_0 = get_bias(n_hidden)
    params += [h_0]
    
    # define the recurrence here
    def step(x_t, h_tm1):
        update_gate = T.nnet.sigmoid(T.dot(x_t, w_in_update) + T.dot(h_tm1, w_hidden_update) + b_update)
        reset_gate = T.nnet.sigmoid(T.dot(x_t, w_in_reset) + T.dot(h_tm1, w_hidden_reset) + b_reset)
        h_t_temp = T.tanh(T.dot(x_t, w_in_hidden) + T.dot(h_tm1 * reset_gate, w_reset_hidden) + b_in_hidden)

        h_t = (1 - update_gate) * h_t_temp + update_gate * h_tm1
        y_t = T.nnet.sigmoid(T.dot(h_t, w_out) + b_out)

        return h_t, y_t

    [_, output], _ = theano.scan(fn=step, sequences=X, outputs_info=[h_0, None], n_steps=X.shape[0])

    test(n_in, n_out, X, output, params)

generate_and_test_gru(10, 50, 1)

Before training: 2.49044999098
200 (lr = 0.100000): 5.0814619226e-06
400 (lr = 0.050000): 3.28566008242e-06
600 (lr = 0.025000): 2.79025599715e-06
800 (lr = 0.012500): 2.59417142659e-06
1000 (lr = 0.006250): 2.5060001764e-06


## Making this a whole lot less painful with Keras

One of the points of going through all these equations was to emphasize how much of a pain it is to do this. We didn't even try stacking layers together! Let's do the exact same thing we did above using Keras, including using an RMSprop optimizer and binary crossentropy cost function.

In [8]:
# parameters
input_dims, output_dims = 10, 2
sequence_length = 20
n_test = 10

# generate some random data to train on
X_data = np.asarray([np.asarray(rng.rand(20, input_dims) > 0.5, dtype=dtype) for _ in range(n_test)])
y_data = np.asarray([np.asarray(rng.rand(output_dims,) > 0.5, dtype=dtype) for _ in range(n_test)])

# put together rnn models
from keras.layers import Input
from keras.layers.recurrent import SimpleRNN, LSTM, GRU
from keras.optimizers import RMSprop
from keras.models import Model

input_sequence = Input(shape=(sequence_length, input_dims,), dtype='float32')

# this is so much easier!
vanilla = SimpleRNN(output_dims, return_sequences=False)
lstm = LSTM(output_dims, return_sequences=False)
gru = GRU(output_dims, return_sequences=False)
rnns = [vanilla, lstm, gru]

# train the models
for rnn in rnns:
    model = Model(input=[input_sequence], output=rnn(input_sequence))
    model.compile(optimizer=RMSprop(lr=0.1), loss='binary_crossentropy')
    print('-- %s --' % rnn.__class__.__name__)
    print('Error before: {}'.format(model.evaluate([X_data], [y_data], verbose=0)))
    model.fit([X_data], [y_data], nb_epoch=1000, verbose=0)
    print('Error after: {}'.format(model.evaluate([X_data], [y_data], verbose=0)))

-- SimpleRNN --


Using Theano backend.


Error before: 4.78796672821
Error after: 0.805904984474
-- LSTM --
Error before: 5.31922245026
Error after: 3.22388005257
-- GRU --
Error before: 3.71665763855
Error after: 1.19209303762e-07
